## This is currently not working

In [88]:
# Install the plaidml backend
import plaidml.keras
plaidml.keras.install_backend()

# Help MacOS be able to use Keras
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [89]:
# better progress bars for jupyter and won't kill model fit
# Command line the following line to get tqdm installed properly
# python -m pip install tqdm
from keras_tqdm import TQDMNotebookCallback
# keras, model definition...
cb = TQDMNotebookCallback()
setattr(cb,'on_train_batch_begin',lambda x,y:None)
setattr(cb,'on_train_batch_end',lambda x,y:None)

In [442]:
from __future__ import print_function
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
import time

In [475]:
batch_size = 32
epochs = 5
# input image dimensions
img_rows, img_cols = 256, 256
input_shape = (img_rows, img_cols, 3)

### Images to Arrays

In [462]:
images = glob.glob('../images/training/build/0/*.tif')
img_list = []
for item in images:
    
    item = np.asarray(Image.open(item))
    item = cv2.resize(item, (img_rows, img_cols))
    item = img_to_array(item) / 255
    item = np.expand_dims(item, axis=0)
    img_list.append(item)

In [463]:
len(img_list)

180

In [464]:
img_list[0].shape

(1, 256, 256, 3)

### Masks to Arrays

In [465]:
masks = glob.glob('../images/training/build_mask/0/*.tif')
mask_list = []
for item in masks:
    
    item = np.asarray(Image.open(item))
    item = cv2.resize(item, (256, 256))
    item = img_to_array(item) / 255
    item = np.expand_dims(item, axis=0)
    mask_list.append(item)

In [466]:
len(mask_list)

180

In [467]:
mask_list[0].shape

(1, 256, 256, 1)

In [429]:
K.image_data_format()

'channels_last'

### One image testing

In [385]:
test_image = cv2.imread('../images/training/build_small/0/vienna31.tif')
test_image = cv2.resize(test_image, (256, 256))
test_image = img_to_array(test_image) / 255
#     item = cv2.cvtColor(item, cv2.COLOR_BGR2GRAY)
    # Doing the not here to convert to RGB from BRG (I think)
#     item = cv2.bitwise_not(item)
# test_image = item.reshape(1, img_rows, img_cols, 3)


In [386]:
test_mask = cv2.imread('../images/training/build_mask_small/0/vienna30.tif')
test_mask = cv2.resize(test_mask, (256, 256))
test_mask = img_to_array(test_mask) / 255
# test_mask = item.reshape(1, img_rows, img_cols, 3)

In [405]:
test_mask

array([[[[1., 1., 1.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)

In [387]:
# This doesn't work for only one image
print(test_image.shape)
test_image = np.expand_dims(test_image, axis=0)
print(test_image.shape)

(256, 256, 3)
(1, 256, 256, 3)


In [388]:
# This doesn't work for only one image
print(test_mask.shape)
test_mask = np.expand_dims(test_mask, axis=0)
print(test_mask.shape)

(256, 256, 3)
(1, 256, 256, 3)


In [468]:
X_train, X_test, y_train, y_test = train_test_split(img_list, mask_list, test_size=.2, random_state=77)

In [469]:
X_train

[array([[[[0.54509807, 0.50980395, 0.4509804 ],
          [0.4       , 0.39607844, 0.36078432],
          [0.27058825, 0.32156864, 0.30980393],
          ...,
          [0.3254902 , 0.3647059 , 0.33333334],
          [0.44313726, 0.46666667, 0.4117647 ],
          [0.33333334, 0.37254903, 0.34117648]],
 
         [[0.30588236, 0.3529412 , 0.32156864],
          [0.37254903, 0.43137255, 0.3647059 ],
          [0.36078432, 0.41568628, 0.3647059 ],
          ...,
          [0.22352941, 0.2901961 , 0.2784314 ],
          [0.2       , 0.22352941, 0.19607843],
          [0.25490198, 0.34509805, 0.32156864]],
 
         [[0.4862745 , 0.5137255 , 0.46666667],
          [0.38039216, 0.41960785, 0.37254903],
          [0.31764707, 0.39607844, 0.34117648],
          ...,
          [0.29803923, 0.37254903, 0.30588236],
          [0.27058825, 0.35686275, 0.3137255 ],
          [0.24705882, 0.32941177, 0.28235295]],
 
         ...,
 
         [[0.34509805, 0.43137255, 0.3529412 ],
          [0.31372

In [375]:
## Not sure I need this

X_train = np.expand_dims(X_train, axis=0)
X_test = np.expand_dims(X_test, axis=0)
y_train = np.expand_dims(y_train, axis=0)
y_test = np.expand_dims(y_test, axis=0)
input_shape = (img_rows, img_cols, 3)
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print('X_test shape:', X_test.shape)
print(X_test.shape[0], 'test samples')
print()
print('y_train shape:', y_train.shape)
print(y_train.shape[0], 'train samples')
print('y_test shape:', y_test.shape)
print(y_test.shape[0], 'test samples')

X_train shape: (1, 204, 256, 3)
1 train samples
X_test shape: (1, 52, 256, 3)
1 test samples

y_train shape: (1, 204, 256, 3)
1 train samples
y_test shape: (1, 52, 256, 3)
1 test samples


### Data Generator For Images and Masks

In [471]:
#start timer
start_time = time.time()

# we create two instances with the same arguments
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     data_format='channels_last')
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 77
image_datagen.fit(test_image, augment=True, seed=seed)
mask_datagen.fit(test_mask, augment=True, seed=seed)

image_generator = image_datagen.flow(
    X_train,
#     class_mode=None,
    seed=seed)

mask_generator = mask_datagen.flow(
    y_train,
#     class_mode=None,
    seed=seed)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)


#End Time
elapsed_time = time.time() - start_time
elapsed_min = elapsed_time / 60
print(f'Total Time: {round(elapsed_min, 3)} minutes.')

Total Time: 0.001 minutes.


In [473]:
mask_generator

In [474]:
mask_generator.labels

AttributeError: 'NumpyArrayIterator' object has no attribute 'labels'

In [481]:

model = Sequential()


model.add(Conv2D(32,
                 kernel_size=(3,3),
                 activation='relu',
                 input_shape=input_shape,
                 data_format='channels_last',
                 padding='same'))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax', name='preds'))
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


#Start timing


start_time = time.time()



model.fit(X_train, 
          y_train, 
          batch_size=batch_size, 
          epochs=epochs,
          steps_per_epoch=len(X_train)//batch_size,
          verbose=0)



# my_gen = ImageDataGenerator(featurewise_center=True,
#                          featurewise_std_normalization=True,
#                          rotation_range=90,
#                          width_shift_range=0.1,
#                          height_shift_range=0.1,
#                          zoom_range=0.2,
#                          data_format='channels_last')




# # Fit model
# model.fit_generator(my_gen.flow(X_train, y_train, batch_size=batch_size),
#            epochs=epochs,
#              # steps per epoch should be total samples / batch size
#             steps_per_epoch=len(X_train)//batch_size,
#               verbose=0)
#            callbacks=[cb])   
#           validation_data=(x_test, y_test)   -- This goes inside the fit
# score = model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

elapsed_time = time.time() - start_time
elapsed_min = elapsed_time / 60
print(f'Total Time: {round(elapsed_min, 3)} minutes.')

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 144 arrays: [array([[[[0.54509807, 0.50980395, 0.4509804 ],
         [0.4       , 0.39607844, 0.36078432],
         [0.27058825, 0.32156864, 0.30980393],
         ...,
         [0.3254902 , 0.3647059 , 0.33333334...

In [457]:
model.history

In [458]:
len(X_train)

5